*The "banknotes.csv" dataset is from kaggle.com*

# Import Necessary Libraries

In [33]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Loading the Data

In [34]:
df = pd.read_csv("banknotes.csv")
df.head()

,conterfeit,Length,Left,Right,Bottom,Top,Diagonal
0,0,214.8,131.0,131.1,9.0,9.7,141.0
1,0,214.6,129.7,129.7,8.1,9.5,141.7
2,0,214.8,129.7,129.7,8.7,9.6,142.2
3,0,214.8,129.7,129.6,7.5,10.4,142.0
4,0,215.0,129.6,129.7,10.4,7.7,141.8


In [35]:
df.describe()

,conterfeit,Length,Left,Right,Bottom,Top,Diagonal
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,0.500000,214.896000,130.121500,129.956500,9.417500,10.650500,140.483500
std,0.501255,0.376554,0.361026,0.404072,1.444603,0.802947,1.152266
min,0.000000,213.800000,129.000000,129.000000,7.200000,7.700000,137.800000
25%,0.000000,214.600000,129.900000,129.700000,8.200000,10.100000,139.500000
50%,0.500000,214.900000,130.200000,130.000000,9.100000,10.600000,140.450000
75%,1.000000,215.100000,130.400000,130.225000,10.600000,11.200000,141.500000
max,1.000000,216.300000,131.000000,131.100000,12.700000,12.300000,142.400000


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   conterfeit  200 non-null    int64  
 1   Length      200 non-null    float64
 2   Left        200 non-null    float64
 3   Right       200 non-null    float64
 4   Bottom      200 non-null    float64
 5   Top         200 non-null    float64
 6   Diagonal    200 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 11.1 KB


#Predictions
> Seperating dataset into training and testing sets

In [37]:
df = df.reindex(np.random.permutation(df.index))

X = df.drop(columns = "conterfeit")
y = df["conterfeit"]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

## Neural Network

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(6),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(1, activation='sigmoid'),

])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train.values, epochs = 50)

Epoch 1/50
5/5 [==============================] - 5s 11ms/step - loss: 0.3345 - accuracy: 0.9000
Epoch 2/50
5/5 [==============================] - 0s 10ms/step - loss: 0.2753 - accuracy: 0.9500
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 0.2297 - accuracy: 0.9625
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1910 - accuracy: 0.9812
Epoch 5/50
5/5 [==============================] - 0s 7ms/step - loss: 0.1601 - accuracy: 0.9812
Epoch 6/50
5/5 [==============================] - 0s 9ms/step - loss: 0.1349 - accuracy: 0.9812
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.1145 - accuracy: 0.9812
Epoch 8/50
5/5 [==============================] - 0s 7ms/step - loss: 0.0995 - accuracy: 0.9812
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 0.0871 - accuracy: 0.9812
Epoch 10/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0768 - accuracy: 0.9875
Epoch 11/50
5/5 [====================

# Predict Values

In [43]:
pred = model.predict(scaler.transform([[0,0,0,0,0,0]]))
pred = [int(round(t)) for t in pred.reshape(1, -1)[0]]
if pred[0] == 1:
  print("FAKE!")
else:
  print("Real :)")

1/1 [==============================] - 0s 51ms/step
FAKE!


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [44]:
pred = model.predict(scaler.transform([[214.8, 131.0, 131.1, 9.0, 9.7, 141.0]]))
pred = [int(round(t)) for t in pred.reshape(1, -1)[0]]
pred[0]
if pred[0] == 1:
  print("FAKE!")
else:
  print("Real :)")

1/1 [==============================] - 0s 28ms/step
Real :)


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# Convert Model to a Downloadable

In [41]:
model.save('/content/model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)